In [46]:
# torch packets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

# torch vision 
from torchvision import datasets, transforms

## prepare  MNIST dataset 

In [47]:
mnist_path = 'DATA'
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])


train_set = datasets.MNIST(root=mnist_path, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=mnist_path, train=False, transform=trans)
train_set_flat = train_set.data.view(60000, 28 * 28 )

## 定义网络

In [48]:
class mnist_net(nn.Module):
    def __init__(self):
        super(mnist_net, self).__init__() # 为啥？
        self.fc1 = nn.Linear(28* 28 ,  500)
        self.fc2 = nn.Linear(500, 32)
        self.fc3 = nn.Linear(32, 10)
        pass
    
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu( self.fc1(x)  )
        x = F.relu( self.fc2(x)  )
        out = self.fc3(x)
        return out 
        pass 
    pass


model = mnist_net()

## 定义 优化器

In [51]:
opt = optim.Adam(model.parameters(), lr= 0.01) # 其他可以使用默认参数

ceriation = nn.CrossEntropyLoss()

batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)


## 训练

In [54]:
TOTAL_EPOCH = 4

for epoch in range(TOTAL_EPOCH):
    ave_loss = 0 # 干什么的？ 
    for batch_index, (x, target) in enumerate(train_loader):
        # 梯度清零
        opt.zero_grad()
        # 计算 输出 
        x, target = Variable(x), Variable(target)
        output = model.forward(x) 
        # print(output)
        # print(output.size())
        loss = ceriation(output , target)
        # print(target)
        # print(target.size())
        ave_loss = ave_loss * 0.9 + loss.item()* 0.1 # 干啥的？
        loss.backward()
        opt.step()
        # print('%4d:  loss: %g'%(batch_index, loss ))
        pass
    print('Epoch: %2d: Loss: %g' %(epoch, ave_loss)) 
    pass

epoch:  0: loss: 0.124059
epoch:  1: loss: 0.11863
epoch:  2: loss: 0.13584
epoch:  3: loss: 0.114831


## 然后是测试 

In [107]:
'''
# 好像是必须要用 test loader 
X_test = Variable(test_set.data.view(-1,1, 28, 28 ))[:96]
print ( X_test.size() )
print( x.size() )
'''
import numpy as np 
from sklearn import  metrics

y_pred = torch.empty(0, dtype= torch.long)
for batch_idx, (x, target) in enumerate(test_loader):
    y_pred_tmp = model.forward(x) 
    label_pred = torch.argmax( y_pred_tmp, dim= 1 )
    #print(label_pred)
    #print(target)
    y_pred = torch.cat( (y_pred, label_pred)  ) # 连接两个向量
    
    pass

y_true = test_set.targets

# 分类结果报告
print(metrics.classification_report(y_true, y_pred)  )

print('\nAccuracy Score: %g' %( metrics.accuracy_score(y_true, y_pred)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.98      0.98      0.98      1135
           2       0.97      0.94      0.96      1032
           3       0.95      0.95      0.95      1010
           4       0.95      0.96      0.96       982
           5       0.98      0.94      0.96       892
           6       0.97      0.97      0.97       958
           7       0.96      0.97      0.96      1028
           8       0.94      0.98      0.96       974
           9       0.96      0.95      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000


Accuracy Score: 0.9634


## 保存模型到本地

In [108]:
torch.save(model.state_dict(), 'mnist_net' )

## 读取保存的模型

In [112]:
the_model = mnist_net()
the_model.load_state_dict(torch.load('mnist_net'))

In [113]:
# 看一下准确率是不是一样的

y_pred2 = torch.empty(0, dtype= torch.long)
for batch_idx, (x, target) in enumerate(test_loader):
    y_pred_tmp = model.forward(x) 
    label_pred = torch.argmax( y_pred_tmp, dim= 1 )
    #print(label_pred)
    #print(target)
    y_pred2 = torch.cat( (y_pred2, label_pred)  ) # 连接两个向量
    
    pass

print('\nAccuracy Score of the Loaded Model: %g' %( metrics.accuracy_score(y_true, y_pred2)))


Accuracy Score: 0.9634
